In [1]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.11:4047
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1633073549378)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


# Example 4.1

This notebook shows Example 4.1 from the book showing how to use SQL on a US Flights Dataset dataset.

Define a UDF to convert the date format into a legible format.

Note: the date is a string with year missing, so it might be difficult to do any queries using SQL year() function

In [3]:
def toDateFormatUDF(dStr:String) : String  = {
  return s"${dStr(0)}${dStr(1)}${'/'}${dStr(2)}${dStr(3)}${' '}${dStr(4)}${dStr(5)}${':'}${dStr(6)}${dStr(7)}"
}

// test  it
toDateFormatUDF("02190925")

toDateFormatUDF: (dStr: String)String
res0: String = 02/19 09:25


Register the UDF

In [4]:
spark.udf.register("toDateFormatUDF", toDateFormatUDF(_:String):String)

res1: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$2059/57260457@4e06328b,StringType,List(Some(class[value[0]: string])),Some(class[value[0]: string]),Some(toDateFormatUDF),true,true)


Read our US departure flight data

In [7]:
val df = spark
  .read
  .format("csv")
  .schema("date STRING, delay INT, distance INT, origin STRING, destination STRING")
  .option("header", "true")
  .option("path", "departuredelays.csv")
  .load()

df

df: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]
res4: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]


Test our UDF

In [8]:
df.selectExpr("toDateFormatUDF(date) as data_format").show(10, false)

+-----------+
|data_format|
+-----------+
|01/01 12:45|
|01/02 06:00|
|01/02 12:45|
|01/02 06:05|
|01/03 12:45|
|01/03 06:05|
|01/04 12:43|
|01/04 06:05|
|01/05 12:45|
|01/05 06:05|
+-----------+
only showing top 10 rows



Create a temporary view to which we can issue SQL queries

In [9]:
df.createOrReplaceTempView("us_delay_flights_tbl")

Convert all date to date_fm so it's more eligible

Note: we are using UDF to convert it on the fly.

In [10]:
spark.sql("SELECT *, date, toDateFormatUDF(date) AS date_fm FROM us_delay_flights_tbl").show(10, false)

+--------+-----+--------+------+-----------+--------+-----------+
|date    |delay|distance|origin|destination|date    |date_fm    |
+--------+-----+--------+------+-----------+--------+-----------+
|01011245|6    |602     |ABE   |ATL        |01011245|01/01 12:45|
|01020600|-8   |369     |ABE   |DTW        |01020600|01/02 06:00|
|01021245|-2   |602     |ABE   |ATL        |01021245|01/02 12:45|
|01020605|-4   |602     |ABE   |ATL        |01020605|01/02 06:05|
|01031245|-4   |602     |ABE   |ATL        |01031245|01/03 12:45|
|01030605|0    |602     |ABE   |ATL        |01030605|01/03 06:05|
|01041243|10   |602     |ABE   |ATL        |01041243|01/04 12:43|
|01040605|28   |602     |ABE   |ATL        |01040605|01/04 06:05|
|01051245|88   |602     |ABE   |ATL        |01051245|01/05 12:45|
|01050605|9    |602     |ABE   |ATL        |01050605|01/05 06:05|
+--------+-----+--------+------+-----------+--------+-----------+
only showing top 10 rows



In [11]:
spark.sql("SELECT COUNT(*) FROM us_delay_flights_tbl").show()

+--------+
|count(1)|
+--------+
| 1391578|
+--------+



### Query 1:

 Find out all flights whose distance between origin and destination is greater than 1000 

In [12]:
spark.sql("SELECT distance, origin, destination FROM us_delay_flights_tbl WHERE distance > 1000 ORDER BY distance DESC").show(10, false)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
+--------+------+-----------+
only showing top 10 rows



A DataFrame equivalent query

In [13]:
df.select("distance", "origin", "destination").where(col("distance") > 1000).orderBy(desc("distance")).show(10, false)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
+--------+------+-----------+
only showing top 10 rows



In [14]:
df.select("distance", "origin", "destination").where($"distance" > 1000).orderBy(desc("distance")).show(10, false)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
|4330    |HNL   |JFK        |
+--------+------+-----------+
only showing top 10 rows



### Query 2:

 Find out all flights with 2 hour delays between San Francisco and Chicago

In [17]:
spark.sql("""
SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC
""").show(10, false)

+--------+-----+------+-----------+
|date    |delay|origin|destination|
+--------+-----+------+-----------+
|02190925|1638 |SFO   |ORD        |
|01031755|396  |SFO   |ORD        |
|01022330|326  |SFO   |ORD        |
|01051205|320  |SFO   |ORD        |
|01190925|297  |SFO   |ORD        |
|02171115|296  |SFO   |ORD        |
|01071040|279  |SFO   |ORD        |
|01051550|274  |SFO   |ORD        |
|03120730|266  |SFO   |ORD        |
|01261104|258  |SFO   |ORD        |
+--------+-----+------+-----------+
only showing top 10 rows



### Query 3:

A more complicated query in SQL, let's label all US flights originating from airports with _high_, _medium_, _low_, _no delays_, regardless of destinations.

In [18]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN  'Long Delays '
                  WHEN delay > 60 AND delay < 120 THEN  'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN   'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'No Delays'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10, false)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|333  |ABE   |ATL        |Long Delays  |
|305  |ABE   |ATL        |Long Delays  |
|275  |ABE   |ATL        |Long Delays  |
|257  |ABE   |ATL        |Long Delays  |
|247  |ABE   |DTW        |Long Delays  |
|247  |ABE   |ATL        |Long Delays  |
|219  |ABE   |ORD        |Long Delays  |
|211  |ABE   |ATL        |Long Delays  |
|197  |ABE   |DTW        |Long Delays  |
|192  |ABE   |ORD        |Long Delays  |
+-----+------+-----------+-------------+
only showing top 10 rows



Some Side Queries

In [19]:
val df1 =  spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")

df1: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 2 more fields]


In [20]:
df1.createOrReplaceGlobalTempView("us_origin_airport_SFO_tmp_view")

In [21]:
val df2 = spark.sql("SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'JFK'")

df2: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 2 more fields]


In [22]:
df2.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [27]:
spark.catalog.listTables(dbName="global_temp").show(truncate=false)

+------------------------------+-----------+-----------+---------+-----------+
|name                          |database   |description|tableType|isTemporary|
+------------------------------+-----------+-----------+---------+-----------+
|us_origin_airport_sfo_tmp_view|global_temp|null       |TEMPORARY|true       |
|us_delay_flights_tbl          |null       |null       |TEMPORARY|true       |
|us_origin_airport_jfk_tmp_view|null       |null       |TEMPORARY|true       |
+------------------------------+-----------+-----------+---------+-----------+

